In [ ]:
# Chase Abram
# IO 2 2021 with Hortacsu

In [1]:
using CSV
using DataFrames
using ForwardDiff
using LinearAlgebra
using Optim

using Random, Distributions
using Statistics

using Plots

In [2]:
# Section 1.3

# Compute the implied probabilities (per logit spec)
function logit_p(p, x, alpha, delta)
    return exp.(alpha*x .+ delta .* reverse(p) .- maximum(p))./(exp(-maximum(p)) .+ exp.(alpha*x .+ delta .* reverse(p) .- maximum(p)))
end

# Finds fixed point (per logit spec)
function fp_logit(x, alpha, delta, sym = false, maxiter = 100, tol = 1e-4)
    if sym
        p = [1/2]
        pnew = [0]
    else
        p = 1/2 .* ones(2)
#         p = [100, 100]
        pnew = zeros(2)
    end
    
    
    f(z) = logit_p(z,x,alpha,delta)
    
    it = 0
    diff = Inf
    
    while it < maxiter && diff > tol
        pnew = f(p)
        diff = abs(maximum(pnew - p))
        it += 1
        p = pnew
        
#         println("it: ", it, ", diff: ", diff, ", p: ", p)
    end
    
#     println("Exited")
#     println("it: ", it, ", diff: ", diff)
#     println("it cond: ", it < maxiter)
#     println("diff cond: ", diff > tol)
    
    if sym
        return p[1]
    end
    
    return p
end



fp_logit (generic function with 4 methods)

In [3]:
alpha = [1, 3]
delta = [1, 6]
x = [1, 2]
p_init = 1/2 .* ones(2)

# logit_p(p_init, x, alpha, delta)

# fp_logit(x, alpha, delta)

for i in 1:2
    for j in 1:2
        println("BNE (sym): alpha = ", alpha[i], ", delta = ", delta[i], ", x = ", x[j])
        println("P_i = ", fp_logit(x[j], alpha[i], delta[i], true))
    end
end
println()
for i in 1:2
    for j in 1:2
        println("BNE: alpha = ", alpha[i], ", delta = ", delta[i], ", x = ", x[j])
        println("P_i = ", fp_logit(x[j], alpha[i], delta[i]))
    end
end

BNE (sym): alpha = 1, delta = 1, x = 1
P_i = 0.8659851186297881
BNE (sym): alpha = 1, delta = 1, x = 2
P_i = 0.9502737662925114
BNE (sym): alpha = 3, delta = 6, x = 1
P_i = 0.9998765126780076
BNE (sym): alpha = 3, delta = 6, x = 2
P_i = 0.9999938555987219

BNE: alpha = 1, delta = 1, x = 1
P_i = [0.8659851186297881, 0.8659851186297881]
BNE: alpha = 1, delta = 1, x = 2
P_i = [0.9502737662925114, 0.9502737662925114]
BNE: alpha = 3, delta = 6, x = 1
P_i = [0.9998765126780076, 0.9998765126780076]
BNE: alpha = 3, delta = 6, x = 2
P_i = [0.9999938555987219, 0.9999938555987219]


In [4]:
alpha = 3
delta = 6
T = 1000
S = 50

Random.seed!(1234)
F = Logistic()
ep = rand(F, S, T, 2)
x = (rand(S, T) .> 1/2) .+ 1
p = fp_logit.(x, alpha, delta, true)
# y = zeros(S,T,2)

y = (alpha .* x .- delta.*p .+ ep) .> 0





50×1000×2 BitArray{3}:
[:, :, 1] =
 1  1  0  1  0  0  0  0  0  0  0  0  1  …  0  0  0  0  0  0  1  1  0  1  0  0
 0  0  1  0  0  0  0  0  0  0  0  1  0     0  1  0  1  0  0  1  0  0  0  0  0
 1  0  1  0  0  1  0  0  1  0  0  0  0     0  0  0  0  0  1  1  0  1  0  1  0
 0  0  0  0  0  0  0  1  0  0  0  0  0     0  0  1  0  0  0  0  0  0  1  0  0
 0  0  0  0  0  0  0  0  0  1  0  0  0     1  0  1  0  0  1  0  0  1  0  0  1
 1  0  0  0  0  0  0  0  0  0  0  0  0  …  0  1  0  0  0  1  1  1  0  0  0  0
 0  1  0  0  0  0  1  0  0  0  0  0  0     0  0  0  0  0  1  0  0  0  0  0  0
 0  0  1  1  1  0  1  0  1  1  1  1  1     1  1  1  0  1  0  0  1  0  1  0  1
 0  0  0  0  0  1  0  1  1  1  1  0  0     0  0  0  1  0  0  0  0  0  0  0  1
 1  0  0  0  0  0  1  0  0  0  1  1  0     0  0  0  1  0  1  0  0  0  0  1  0
 0  1  0  0  0  0  0  1  0  0  0  0  0  …  1  0  0  0  1  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  1  0  1  0     0  0  1  0  0  0  0  0  1  0  0  0
 0  1  1  0  0  0  0  0  1  0

In [5]:
# Load data
df = DataFrame()
df = CSV.read("psetTwo.csv", DataFrame)

# Mileage
mileage = df.milage
N = length(mileage)

# Find replacement periods
d_rep = [0; (mileage[2:N] - mileage[1:N-1]) .< 0]

5000-element Array{Int64,1}:
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 ⋮
 0
 1
 0
 0
 0
 0
 0
 0
 0
 0
 0
 1

In [257]:
# Discretize the domain (equispacing)
# function disc_domain(data, K)
#     M = maximum(data)
#     m = minimum(data)
#     return LinRange(m,M,K)
# end

# Discretize domain (equal mass in each bin)
function disc_domain(data, K)
    
    # Sort data
    data_sorted = sort(data)
    
    # Initialize
    disc = zeros(K)
    
    # Chunk up so each bin has same number of entries
    for k in 1:K
        disc[k] = data_sorted[Int(floor(k*N/K))]
    end
    
    return disc
end

# Map value to its chunk
function to_chunk(vs, chunks)
    
    # Initialize
    its = zeros(length(vs))
    
    # Find chunk for each v
    for j in 1:length(vs)
        it = 1
        
        # Continue until value larger than lower end of chunk
        while vs[j] > chunks[it] && it < length(chunks)
            it += 1
        end
        its[j] = it
    end
    return Int64.(its)
end

# Create transition matrices
function trans_mats(x, d, K)
    
    # Get discretized domain
    disc = disc_domain(x, K)
    
    # For replacement
#     rep = zeros(K,K)
    
    # For no replacement
    no_rep = zeros(K,K)
    
    # Get chunks
    chunk_map = to_chunk(x, disc)
    
    # Fix this for single matrix
#     for i in 2:length(m)
#         if d[i] > 0
#             rep[chunk_map[i-1], chunk_map[i]] += 1
#         else
#             no_rep[chunk_map[i-1], chunk_map[i]] += 1
#         end
#     end
    
    # Add mass in each transition
    for i in 2:length(x)
        no_rep[chunk_map[i-1], chunk_map[i]] += 1
    end
    
    # If no entries in row, assume full mass on diag?
    for k in 1:K
#         if sum(rep[k,:]) == 0
#             rep[k,1] = 1
#         end
        
        if sum(no_rep[k,:]) == 0
            no_rep[k,k] = 1
        end
        
    end
    
    # Normalize
#     rep = rep.*(1 ./max.(sum(rep,dims=2), 1.0))
    no_rep = no_rep.*(1 ./sum(no_rep,dims=2))
    
    return no_rep #, rep
end

# Utility (shock not included)
function u(x, d, theta)
    if d == 0
        return -theta[1].*x - theta[2].*(x./100).^2
    else
        return -theta[3].*ones(length(x))
    end
end

# Jacobian of u wrt theta
function uj(x, d, theta)
    if d == 0
        return hcat(-x, -(x./100).^2, zeros(length(x)))
    else
        return hcat(zeros(length(x)), zeros(length(x)), -ones(length(x)))
    end
end


# For fixed point of EV
function Gamma(EV, P, X, beta, theta)
    return P*log.(exp.(u(X,0,theta) .+ beta.*EV) .+ exp.(u(X,1,theta) .+ beta.*EV[1]))
end

# Conditional choice probs
# (sum to 1 when considering exactly exhaustive choices)
# Default to just using X, EV
function ch_probs(EV, X, beta, theta, x_inds = [i for i in 1:length(X)])
    return exp.(u(X[x_inds],0,theta) .+ beta.*EV[x_inds] .- maximum(u(X[x_inds],0,theta) .+ beta.*EV[x_inds]))./(exp.(u(X[x_inds],0,theta) .+ beta.*EV[x_inds] .- maximum(u(X[x_inds],0,theta) .+ beta.*EV[x_inds])) .+ exp.(u(X[x_inds],1,theta) .+ beta.*EV[1] .- maximum(u(X[x_inds],0,theta) .+ beta.*EV[x_inds])))
end

# Jacobian of Gamma wrt EV (K x K)
function Gamma_jac_EV(EV, P, X, beta, theta)
    return P*(diagm(ch_probs(EV, X, beta, theta)) .* beta .+ (1 .- ch_probs(EV, X, beta, theta)) .* beta .* [i == 1 for j in 1:length(EV), i in 1:length(EV)])
end

# Jacobian of Gamma wrt theta (K x 3)
function Gamma_jac_theta(EV, P, X, beta, theta)
    return P*(exp.(u(X,0,theta) .+ beta.*EV .- maximum(u(X,0,theta) .+ beta.*EV))  .* uj(X,0,theta) .+ exp.(u(X,1,theta) .+ beta.*EV[1] .- maximum(u(X,0,theta) .+ beta.*EV)) .* uj(X,1,theta))./(exp.(u(X,0,theta) .+ beta.*EV .- maximum(u(X,0,theta) .+ beta.*EV)) .+ exp.(u(X,1,theta) .+ beta.*EV[1] .- maximum(u(X,0,theta) .+ beta.*EV)))
end

# Contraction mapping to solve EV
function contract_EV(EV, P, X, beta, theta, maxiter = 1000, tol = 1e-5)
    
    # Initialize
    it = 1
    diff = Inf
    EVnew = zeros(length(EV))
    
    while it < maxiter && diff > tol
        
        # Find new
        EVnew = Gamma(EV, P, X, beta, theta)
        
        # Find diff
        diff = maximum(abs.(EVnew - EV))
        
        # Update
        it += 1
        EV = EVnew
    end
    
    # Status updates
    println("Contraction done")
    println("    it: ", it)
    println("    diff: ", diff)
    println("    EV: ", EV)
    
    return EV
end

# Newton-Kantorovich to solve EV
function nk_EV(EV, P, X, beta, theta, maxiter = 1000, tol = 1e-14)
    
    # Initialize
    it = 1
    diff = Inf
    EVnew = zeros(length(EV))
    
    while it < maxiter && diff > tol
        
        # Get new
        EVnew = EV - (I - Gamma_jac_EV(EV, P, X, beta, theta)) \ (EV - Gamma(EV, P, X, beta, theta))
        
        # Get diff
        diff = maximum(abs.(EVnew - EV))
        
        # Update
        it += 1
        EV = EVnew
    end
    
    # Status updates
    println("Newton-Kantorovich done")
    println("    it: ", it)
    println("    diff: ", diff)
    println("    EV: ", EV)
    
    return EV
end

# Combine contraction and Newton-Kantorovich
function poly_algo(P, X, beta, theta,
        maxiter_c = 1000, tol_c = 1e-5,
        maxiter_nk = 1000, tol_nk = 1e-14
    )
    
    # Initialize
    EV = zeros(length(X))
    
    # Contract
    EV = contract_EV(EV, P, X, beta, theta, maxiter_c, tol_c)
    
    # NK
    EV = nk_EV(EV, P, X, beta, theta, maxiter_nk, tol_nk)
    
    return EV
end

# Log-likelihood
function log_lik(theta, beta, d, X, EV, x_inds)
    return d'*log.(1 .- ch_probs(EV, X, beta, theta, x_inds)) .+ (1 .- d')*log.(ch_probs(EV, X, beta, theta, x_inds))
end

# Expression inside exp in jacobian of probs
# Note: no - max(V) stabilization here
function inside_exp(theta, beta, X, EV, x_inds)
    return u(X[x_inds],1,theta) .+ beta.* EV[1] .- u(X[x_inds],0,theta) .- beta.*EV[x_inds]
end

# Jacobian of choice probs wrt theta
function p_jac(theta, beta, X, EV, EV_jac, x_inds = [i for i in 1:length(X)])
    return (uj(X[x_inds],1,theta) .+ beta.* repeat(EV_jac[1,:]', size(EV_jac[x_inds],1)) .- uj(X[x_inds],0,theta) .- beta.*EV_jac[x_inds]) .* -exp.(inside_exp(theta, beta, X, EV, x_inds))./(1 .+ exp.(inside_exp(theta, beta, X, EV, x_inds))).^2
end

# Jacobian of EV wrt theta
function EV_jac(EV, P, X, beta, theta)
    return (I - Gamma_jac_EV(EV, P, X, beta, theta)) \ Gamma_jac_theta(EV, P, X, beta, theta)
end

# Jacobian of log likelihood (wrt theta)
function log_lik_jac(theta, beta, d, X, EV, P, x_inds)
    return p_jac(theta, beta, X, EV, EV_jac(EV, P, X, beta, theta), x_inds)' * (-d.*(1 ./(1 .- ch_probs(EV, X, beta, theta, x_inds))) .+ (1 .- d')*(1 ./ch_probs(EV, X, beta, theta, x_inds)))
end

log_lik_jac (generic function with 2 methods)

In [262]:
Random.seed!(1234)
K = 5
P = trans_mats(mileage, d_rep, K)
X = disc_domain(mileage, K)
D = rand(K) .> 1/2
beta = 0.99
theta = [0.01,0.02,0.03]

x = mileage
x_inds = to_chunk(x, X)
d = d_rep

EV = zeros(K)

# ch_probs(EV, X, beta, theta)

# contract_EV(EV, P, X, beta, theta)
# nk_EV(EV, P, X, beta, theta)

EV = poly_algo(P, X, beta, theta, 100, 1e-4, 100, 1e-14)
# EV[x_ind]

# d = [1.0, 0.0, 1.0, 0.0, 0.0]
# x = [X[1], X[2], X[1], X[2], X[3]]

# uj(X, 0, theta)
# uj(X, 1, theta)
# Gamma_jac_theta(EV, P, X, beta, theta)
# EV_jac(EV, P, X, beta, theta)

# log_lik(theta, beta, d, X, EV, x_inds)




# Check Jacobians against AD here

# u
# println("u jac: ", uj(x, 0, theta))
# u_forAD(z) = u(x, 0, z)
# ForwardDiff.jacobian(u_forAD, theta) - uj(x, 0, theta)

# Gamma EV
# println("Gamma jac EV: ", Gamma_jac_EV(EV, P, X, beta, theta))
# GEV_forAD(z) = Gamma(z, P, X, beta, theta)
# ForwardDiff.jacobian(GEV_forAD, EV) - Gamma_jac_EV(EV, P, X, beta, theta)

# Gamma theta
println("Gamma jac theta: ", Gamma_jac_EV(EV, P, X, beta, theta))
# GEV_forAD(z) = Gamma(z, P, X, beta, theta)
# ForwardDiff.jacobian(GEV_forAD, EV) - Gamma_jac_EV(EV, P, X, beta, theta)





# println("ll jac: ", log_lik_jac(theta, beta, d, X, EV, P, x_inds))
# ll(z) = log_lik(z, beta, d, X, EV, x_inds)

# ForwardDiff.jacobian(ll, theta)

Contraction done
    it: 100
    diff: 0.18156887370862052
    EV: [30.66751158091334, 30.535551071741004, 30.448137270876735, 30.373014490081893, 30.36280498046687]
Newton-Kantorovich done
    it: 3
    diff: 0.0
    EV: [48.64283007806667, 48.51086956889434, 48.42345576803007, 48.34833298723523, 48.3381234776202]
Gamma jac theta: [0.85189544097681 0.13810455902319 0.0 0.0 0.0; 0.6772321322307928 0.2026291287979797 0.11013873897122756 0.0 0.0; 0.7520895321792236 0.0 0.16585440504754176 0.07205606277323463 0.0; 0.8432763336518263 0.0 0.0 0.12730174303746647 0.019421923310707318; 0.93948799495243 0.0 0.0 0.0 0.05051200504757009]


In [132]:
K = 5

disc = disc_domain(mileage, K)

# to_chunk(mileage, disc)

# # plot(to_chunk(mileage, disc))

trans_mats(mileage, d_rep, K)[2]

5×5 Array{Float64,2}:
 0.599607  0.400393  0.0       0.0       0.0
 0.0       0.590447  0.409553  0.0       0.0
 0.0       0.0       0.635551  0.364449  0.0
 0.0       0.0       0.0       0.690722  0.309278
 0.0       0.0       0.0       0.0       1.0